<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Parts-of-Speech-Tagging-Begginer/blob/main/NLP_Parts_of_Speech_Tagging_Begginer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import os
import re
import collections
import numpy as np
import pandas as pd

In [3]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_24.pos

--2022-04-15 12:00:45--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_24.pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286063 (279K) [text/plain]
Saving to: ‘WSJ_24.pos.1’

WSJ_24.pos.1        100%[===================>] 279.36K  --.-KB/s    in 0.02s   

2022-04-15 12:00:45 (12.4 MB/s) - ‘WSJ_24.pos.1’ saved [286063/286063]



In [4]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_02-21.pos

--2022-04-15 12:00:46--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_02-21.pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8279089 (7.9M) [text/plain]
Saving to: ‘WSJ_02-21.pos.1’

WSJ_02-21.pos.1     100%[===================>]   7.90M  --.-KB/s    in 0.07s   

2022-04-15 12:00:46 (115 MB/s) - ‘WSJ_02-21.pos.1’ saved [8279089/8279089]



In [5]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/hmm_vocab.txt

--2022-04-15 12:03:04--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/hmm_vocab.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196571 (192K) [text/plain]
Saving to: ‘hmm_vocab.txt’

hmm_vocab.txt       100%[===================>] 191.96K  --.-KB/s    in 0.02s   

2022-04-15 12:03:04 (10.4 MB/s) - ‘hmm_vocab.txt’ saved [196571/196571]



In [6]:
with open('WSJ_02-21.pos', 'r') as f:
  train_corpus = f.readlines()

In [8]:
train_corpus[10: 20]

['at\tIN\n',
 'Chicago\tNNP\n',
 "'s\tPOS\n",
 'Goodman\tNNP\n',
 'Theatre\tNNP\n',
 '(\t(\n',
 '``\t``\n',
 'Revitalized\tVBN\n',
 'Classics\tNNS\n',
 'Take\tVBP\n']

In [9]:
with open('WSJ_24.pos', 'r') as f:
  y = f.readlines()

In [10]:
y[10: 20]

['points\tNNS\n',
 'this\tDT\n',
 'week\tNN\n',
 ',\t,\n',
 'with\tIN\n',
 'readings\tNNS\n',
 'on\tIN\n',
 'trade\tNN\n',
 ',\t,\n',
 'output\tNN\n']

In [20]:
with open('hmm_vocab.txt', 'r') as f:
  vocabulary_list = f.read().split('\n')

vocab = {}
for ind, word in enumerate(sorted(vocabulary_list)):
  vocab[word] = ind

In [21]:
sorted(vocabulary_list)[10: 20]

["'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m"]

In [27]:
for id, (k, v) in enumerate(vocab.items()):
  print(k, v)
  if id == 10:
    break

 0
! 1
# 2
$ 3
% 4
& 5
' 6
'' 7
'40s 8
'60s 9
'70s 10


In [76]:
# https://github.com/amanjeetsahu/Natural-Language-Processing-Specialization/blob/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/utils_pos.py

import string


# Punctuation characters
punct = set(string.punctuation)

# Morphology rules used to assign unknown word tokens
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]



def word_tag_spliter(word_tag, vocab):

  if word_tag.split():
    word, tag = word_tag.split()
    if not vocab.get(word):
      word = assign_unk(word)
  else:
      word, tag = '--n--', '--s--'
  
  return word, tag

def assign_unk(tok):
    """
    Assign unknown word tokens
    """
    # Digits
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Punctuation
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Upper-case
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Nouns
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Verbs
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Adjectives
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Adverbs
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"

def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    with open(data_fp, "r") as data_file:
        for cnt, word in enumerate(data_file):
            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


In [29]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/test.words

--2022-04-15 12:43:47--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/test.words
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180266 (176K) [text/plain]
Saving to: ‘test.words’

test.words          100%[===================>] 176.04K  --.-KB/s    in 0.01s   

2022-04-15 12:43:47 (11.6 MB/s) - ‘test.words’ saved [180266/180266]



In [77]:
_, prep = preprocess(vocab, 'test.words')

In [78]:
prep[10:20]

['points',
 'this',
 'week',
 ',',
 'with',
 'readings',
 'on',
 'trade',
 ',',
 'output']

In [79]:
from collections import defaultdict

def dictionary_creator(corpus, vocab):
  emission_counts = defaultdict(int)
  transition_counts = defaultdict(int)
  tag_counts = defaultdict(int)
  prev_tag = '--s--'
  for word_tag in corpus:
    word, tag = word_tag_spliter(word_tag, vocab)

    transition_counts[(prev_tag, tag)] += 1
    emission_counts[(tag, word)] += 1
    tag_counts[tag] += 1
    prev_tag = tag
  
  return emission_counts, transition_counts, tag_counts 

In [80]:
emission_counts, transition_counts, tag_counts = dictionary_creator(train_corpus, vocab)

In [83]:
emission_counts

defaultdict(int,
            {('IN', 'In'): 1735,
             ('DT', 'an'): 3142,
             ('NNP', 'Oct.'): 317,
             ('CD', '19'): 100,
             ('NN', 'review'): 36,
             ('IN', 'of'): 22925,
             ('``', '``'): 6967,
             ('DT', 'The'): 6795,
             ('NN', 'Misanthrope'): 3,
             ("''", "''"): 6787,
             ('IN', 'at'): 4361,
             ('NNP', 'Chicago'): 197,
             ('POS', "'s"): 8079,
             ('NNP', 'Goodman'): 7,
             ('NNP', 'Theatre'): 5,
             ('(', '('): 1153,
             ('VBN', '--unk_upper--'): 93,
             ('NNS', '--unk_upper--'): 350,
             ('VBP', 'Take'): 1,
             ('DT', 'the'): 41098,
             ('NN', 'Stage'): 1,
             ('IN', 'in'): 14957,
             ('NNP', '--unk_upper--'): 4635,
             ('NNP', 'City'): 132,
             (',', ','): 48723,
             ('NN', 'Leisure'): 2,
             ('CC', '&'): 1033,
             ('NNS', 'Arts'): 2,


In [84]:
transition_counts

defaultdict(int,
            {('--s--', 'IN'): 5050,
             ('IN', 'DT'): 32364,
             ('DT', 'NNP'): 9044,
             ('NNP', 'CD'): 1752,
             ('CD', 'NN'): 7377,
             ('NN', 'IN'): 32885,
             ('IN', '``'): 546,
             ('``', 'DT'): 1014,
             ('DT', 'NN'): 38873,
             ('NN', "''"): 686,
             ("''", 'IN'): 591,
             ('IN', 'NNP'): 14753,
             ('NNP', 'POS'): 5094,
             ('POS', 'NNP'): 905,
             ('NNP', 'NNP'): 34465,
             ('NNP', '('): 313,
             ('(', '``'): 42,
             ('``', 'VBN'): 75,
             ('VBN', 'NNS'): 759,
             ('NNS', 'VBP'): 5076,
             ('VBP', 'DT'): 1473,
             ('NNP', ','): 12688,
             (',', "''"): 2796,
             ("''", 'NN'): 242,
             ('NN', 'CC'): 5369,
             ('CC', 'NNS'): 1714,
             ('NNS', ')'): 98,
             (')', ','): 225,
             (',', 'DT'): 6487,
             (',', '

In [85]:
tag_counts

defaultdict(int,
            {'#': 142,
             '$': 7372,
             "''": 6919,
             '(': 1366,
             ')': 1376,
             ',': 48727,
             '--s--': 39832,
             '.': 39478,
             ':': 4772,
             'CC': 23947,
             'CD': 36568,
             'DT': 81842,
             'EX': 863,
             'FW': 234,
             'IN': 98554,
             'JJ': 61217,
             'JJR': 3238,
             'JJS': 1947,
             'LS': 36,
             'MD': 9803,
             'NN': 132935,
             'NNP': 91466,
             'NNPS': 2673,
             'NNS': 59856,
             'PDT': 370,
             'POS': 8701,
             'PRP': 17436,
             'PRP$': 8407,
             'RB': 30970,
             'RBR': 1768,
             'RBS': 451,
             'RP': 2662,
             'SYM': 58,
             'TO': 22357,
             'UH': 97,
             'VB': 26438,
             'VBD': 29889,
             'VBG': 14846,
             '